In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%%time
import re
import random
import textwrap
from neo4j import GraphDatabase

ZMQInteractiveShell_obj = get_ipython()
uri = 'bolt://localhost:7687'
user = 'neo4j'
password = 'Genesis11'
driver = GraphDatabase.driver(uri, auth=(user, password))

Wall time: 2.24 s


In [3]:

import pandas as pd

def do_cypher_tx(tx, cypher):
    results_list = tx.run(cypher)
    values_list = []
    for record in results_list:
        values_list.append(dict(record.items()))
    
    return values_list

In [6]:

# Get the distinct node labels and their counts
tx_str = 'MATCH (n) RETURN DISTINCT LABELS(n) AS label_name, COUNT(*) AS label_count'
with driver.session() as session:
    rows_list = session.read_transaction(do_cypher_tx, tx_str)
labels_df = pd.DataFrame(rows_list)
# labels_df.label_name = labels_df.label_name.map(lambda x: x[0])
# labels_df = labels_df.set_index('label_name')
labels_df

""


In [15]:

# Get the distinct relationship types and their counts
tx_str = 'MATCH (a)-[r]-(b) RETURN DISTINCT TYPE(r) AS type_name, COUNT(*) AS type_count'
with driver.session() as session:
    rows_list = session.read_transaction(do_cypher_tx, tx_str)
types_df = pd.DataFrame(rows_list)
types_df = types_df.set_index('type_name')
types_df

,type_count
type_name,
IS_CONTAINED_IN,107824
IS_PART_OF,107824
SUMMARIZES,76602


In [22]:

# Get the incomplete child strings labeled in the minimum requirements column
r = '(fn:FileNames)-[file_name_id]-(nps:NavigableParentSequence)-[navigable_parent_id]-(np:NavigableParents)'
tx_str = f'''
    MATCH {r}
    RETURN
        fn.file_name AS file_name, fn.file_name_id AS file_name_id,
        nps.mrs_id AS mrs_id, nps.navigable_parent_sequence_id AS navigable_parent_sequence_id,
        nps.sequence_order AS sequence_order,
        np.header_tag_id AS header_tag_id, np.navigable_parent_id AS navigable_parent_id,
        np.navigable_parent AS navigable_parent;'''
with driver.session() as session:
    rows_list = session.read_transaction(do_cypher_tx, tx_str)
nps_df = pd.DataFrame(rows_list)
nps_df.sample(3).T

,6995,5408,2280
file_name,Data_Scientist_690af6480b6c96e4.html,"Data_Scientist_-_New_York,_NY_10016_-_Indeed.c...",Data_Engineer_III_-_Colorado_-_Indeed.com_782f...
file_name_id,177,135,50
mrs_id,0,0,0
navigable_parent_sequence_id,10123,7455,3031
sequence_order,60,8,21
header_tag_id,67,88,67
navigable_parent_id,9999,14108,7103
navigable_parent,<li>Retirement Plan</li>,<p>· Deploy machine learning across the entire...,<li>Experience with agile development processe...


In [23]:

mask_series = (nps_df.mrs_id != 0)
file_name_ids_list = nps_df[mask_series].file_name_id.unique().tolist()
mask_series = nps_df.file_name_id.isin(file_name_ids_list)
CHILD_STRS_LIST = nps_df[mask_series].navigable_parent.tolist()

In [48]:

# Get all possible unlabeled HTML strings from the database
r = '(mrs:MinimumRequirementsSection)-[mrs_id]-(nps:NavigableParentSequence)-[navigable_parent_id]-'
r += '(np:NavigableParents)'
APU_TX_STR = f'''
    MATCH {r}
    WHERE
        (np.is_task_scope IS NULL) OR
        (np.is_minimum_qualification IS NULL) OR
        (np.is_preferred_qualification IS NULL) OR
        (np.is_legal_notification IS NULL) OR
        (np.is_job_title IS NULL) OR
        (np.is_office_location IS NULL) OR
        (np.is_job_duration IS NULL) OR
        (np.is_supplemental_pay IS NULL) OR
        (np.is_educational_requirement IS NULL) OR
        (np.is_interview_procedure IS NULL) OR
        (np.is_corporate_scope IS NULL) OR
        (np.is_posting_date IS NULL) OR
        (np.is_other IS NULL)
    RETURN
        np.navigable_parent AS navigable_parent,
        mrs.mrs_explanation AS mrs_explanation;'''
print(TX_STR)


    MATCH (mrs:MinimumRequirementsSection)-[mrs_id]-(nps:NavigableParentSequence)-[navigable_parent_id]-(np:NavigableParents)
    WHERE
        (np.is_task_scope IS NULL) OR
        (np.is_minimum_qualification IS NULL) OR
        (np.is_preferred_qualification IS NULL) OR
        (np.is_legal_notification IS NULL) OR
        (np.is_job_title IS NULL) OR
        (np.is_office_location IS NULL) OR
        (np.is_job_duration IS NULL) OR
        (np.is_supplemental_pay IS NULL) OR
        (np.is_educational_requirement IS NULL) OR
        (np.is_interview_procedure IS NULL) OR
        (np.is_corporate_scope IS NULL) OR
        (np.is_posting_date IS NULL) OR
        (np.is_other IS NULL)
    RETURN
        np.navigable_parent AS navigable_parent,
        mrs.mrs_explanation AS mrs_explanation;


In [98]:

def get_tx_code():
    with driver.session() as session:
        rows_list = session.read_transaction(do_cypher_tx, APU_TX_STR)
    nonheader_quals_df = pd.DataFrame(rows_list)
    mask_series = nonheader_quals_df.navigable_parent.isin(CHILD_STRS_LIST)
    output_str = ''
    df = nonheader_quals_df[mask_series]
    records_count = df.shape[0]
    output_str += f'\n# Only {records_count:,} records to go\n'
    if records_count:
        df = df.sample(1)
        output_str += f"# This tag is labeled as {df.mrs_explanation.squeeze()}\ntag_str =  '"
        tag_str = df.navigable_parent.squeeze().replace("'", "\\'")
        output_str += "'\ntag_str += '".join(textwrap.wrap(tag_str, expand_tabs=False,
                                                           replace_whitespace=False,
                                                           break_long_words=False, drop_whitespace=False))
        output_str += "'\n"
        output_str += 'tx_str = "MATCH (np:NavigableParents {navigable_parent: \'" + tag_str + r"""\'})\n'
        output_str += 'SET\n    np.is_header = 0, np.is_task_scope = 0,\n'
        output_str += '    np.is_minimum_qualification = 0, np.is_preferred_qualification = 0,\n'
        output_str += '    np.is_legal_notification = 0, np.is_job_title = 0,\n'
        output_str += '    np.is_office_location = 0, np.is_job_duration = 0,\n'
        output_str += '    np.is_supplemental_pay = 0, np.is_educational_requirement = 0,\n'
        output_str += '    np.is_interview_procedure = 0, np.is_corporate_scope = 0,\n'
        output_str += '    np.is_posting_date = 0, np.is_other = 0\n'
        output_str += 'RETURN PROPERTIES(np) AS properties;"""\n'
        output_str += 'with driver.session() as session:\n'
        output_str += "        row_dict = session.write_transaction(do_cypher_tx, tx_str)[0]['properties']\n"
        output_str += 'pd.DataFrame([row_dict]).T'
    
    return output_str

In [ ]:

ZMQInteractiveShell_obj.set_next_input(text=get_tx_code(), replace=True)

In [146]:

# Only 19 records to go
# This tag is labeled as Not a part of the Minimum Requirements Section
tag_str =  '<li>Using the FNA platform and scripting language, identify hidden '
tag_str += 'behavioural patterns and interconnections in large datasets, helping '
tag_str += 'to create breakthrough solutions, performing exploratory and targeted '
tag_str += 'data analyses as part of quantitative services engagements or proof of'
tag_str += ' concepts. This will be achieved both remotely and on-site with '
tag_str += 'clients</li>'
tx_str = "MATCH (np:NavigableParents {navigable_parent: '" + tag_str + r"""'})
SET
    np.is_header = 0, np.is_task_scope = 0,
    np.is_minimum_qualification = 0, np.is_preferred_qualification = 0,
    np.is_legal_notification = 0, np.is_job_title = 0,
    np.is_office_location = 0, np.is_job_duration = 0,
    np.is_supplemental_pay = 0, np.is_educational_requirement = 0,
    np.is_interview_procedure = 0, np.is_corporate_scope = 0,
    np.is_posting_date = 0, np.is_other = 0
RETURN PROPERTIES(np) AS properties;"""
with driver.session() as session:
        row_dict = session.write_transaction(do_cypher_tx, tx_str)[0]['properties']
pd.DataFrame([row_dict]).T